In [2]:
import pandas as pd

In [55]:
import numpy as np

In [3]:
import os

In [13]:
for file in os.listdir('../data/'):
    print(file + '\t\t\t\t\t\t' + str(os.stat("../data/" + file).st_size/1000000))

clicks.csv.gzip						4.248096
auctions.csv.gzip						700.836701
Recomendaciones y aclaraciones.docx						0.006909
installs.csv.gzip						22.758788
target_final_competencia_revamped.csv						0.147415
events.csv.gzip						441.00504


# Clicks

In [82]:
clicks_dtypes = {'advertiser_id':'category',
                 'action_id':'category',
                 'source_id':'category',
                 'country_code':'category',
                 'latitude':'float64',
                 'longitude':'float64',
                 'wifi_connection':'bool',
                 'carrier_id':'category',
                 'trans_id':'category',
                 'os_minor':'category',
                 'agent_device':'category',
                 'os_major':'category',
                 'specs_brand':'category',
                 'brand':'category',
                 'timeToClick':'float64',
                 'touchX':'object',
                 'touchY':'object',
                 'ref_type':'category',
                 'ref_hash':'category'}

df_clicks = pd.read_csv('../data/clicks.csv.gzip', compression='gzip', dtype=clicks_dtypes, parse_dates=['created'])

# Eliminar valor "Infinity" de touchX y touchY, y dejar datos como floats.
df_clicks.loc[df_clicks['touchX'] == 'Infinity', 'touchX'] = np.inf

df_clicks.loc[df_clicks['touchY'] == 'Infinity', 'touchY'] = np.inf

df_clicks['touchX'] = df_clicks['touchX'].astype('float32')

df_clicks['touchY'] = df_clicks['touchY'].astype('float32')

In [84]:
df_clicks.head()

,advertiser_id,action_id,source_id,created,country_code,latitude,longitude,wifi_connection,carrier_id,trans_id,os_minor,agent_device,os_major,specs_brand,brand,timeToClick,touchX,touchY,ref_type,ref_hash
0,1,NaN,2,2019-04-18 05:27:42.197000+00:00,6287817205707153877,1.714547,0.871535,False,3.0,9JMAfrb-b9cSEVCJb0P9JfihGthaS7E,1.5176438893491397e+18,NaN,5.131615556736863e+18,71913840936116953,0.0,2.317,0.968,0.503,1891515180541284343,1293710398598742392
1,1,NaN,1,2019-04-18 05:27:03.164000+00:00,6287817205707153877,1.714512,0.871062,True,2.0,r3xtTRv2lInfiXG8JI3NQsNcBo8GyFQ,1.2885781261232225e+18,NaN,3.908390200756879e+18,3576558787748411622,1.0,7.653,0.712,1.689,1891515180541284343,1663930990551616564
2,1,NaN,1,2019-04-18 05:42:07.926000+00:00,6287817205707153877,1.714547,0.871535,True,4.0,WOnHFqQtY48z_ygKZ-030U_g0TMGVMw,2.2387361390161664e+18,NaN,3.581232574980917e+18,3576558787748411622,NaN,464.796,0.227,0.251,1891515180541284343,8488038938665586188
3,1,NaN,1,2019-04-18 05:26:04.446000+00:00,6287817205707153877,1.708041,0.870772,True,1.0,wQMLLmYqiFhSuha9p9B13PMtcyBW_vM,2.411639939918313e+18,NaN,3.908390200756879e+18,3576558787748411622,NaN,225.311,0.696,6.587,1891515180541284343,6488361690105189959
4,1,NaN,1,2019-04-18 05:23:37.764000+00:00,6287817205707153877,1.715514,0.870772,True,2.0,GeFoyBzMA7taylMxxjzlNPTU-n4FXFs,1.5176438893491397e+18,NaN,5.131615556736863e+18,3576558787748411622,0.0,84.736,0.059,0.142,1891515180541284343,1348993302102753419
